# **Episode II: Attack of the YOLOv5**

## 0. Preambule 

here we choose to work with YOLOv5 with a docker container, to pull the image and start a container use the following command
```
t=ultralytics/yolov5:latest && docker pull $t && docker run -it --ipc=host --gpus all -p 8888:8888 -v $PWD:/usr/src/app/AIpprenticeChronicles $t
```
then start jupyter from the container using the command
```
jupyter notebook --ip 0.0.0.0 --no-browser --port=8888 --allow-root

```
and open the notebook `AIpprentice_chronicles_episode2.ipynb`.

## 1. Introduction

YOLOv5 is a state-of-the-art object detection model known for its speed and accuracy. Built upon the You Only Look Once (YOLO) concept, YOLOv5 introduces a streamlined architecture consisting of a backbone network, neck network, and detection head. It is trained on large-scale datasets like COCO and utilizes anchor boxes for bounding box predictions. YOLOv5 leverages advanced techniques such as multi-scale training, data augmentation, and focal loss to improve object detection performance. With its efficient architecture and comprehensive training pipeline, YOLOv5 has become a popular choice for real-time object detection tasks.

Transfer learning is a powerful technique in deep learning where a pre-trained model, typically trained on a large-scale dataset, is utilized as a starting point for a new task or dataset. By leveraging the knowledge learned from the pre-training phase, transfer learning enables the transfer of valuable representations and learned features to the new task.

This notebook largely borrows from the nice article [YOLOv5 Transfer Learning In Simple Steps Without Losing Your Mind](https://kikaben.com/yolov5-transfer-learning-dogs-cats/)

## 2. Data Preparation

Using the scripts implemented in the first episode we generate a dataset containing 1000 labelled images and display the first 100.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm 
from PIL import Image, ImageDraw
import shutil
import glob

# Define a dictionary to store information about different objects
objects = {
    0: {'name': 'Sun', 'file': 'sun.png', 'code': 'SU'},
    1: {'name': 'Earth', 'file': 'earth.png', 'code': 'EA'},
    2: {'name': 'Mars', 'file': 'mars.png', 'code': 'MA'},
    3: {'name': 'Venus', 'file': 'venus.png', 'code': 'VE'},
    4: {'name': 'Jupiter', 'file': 'jupiter.png', 'code': 'JU'},
    5: {'name': 'Mercury', 'file': 'mercury.png', 'code': 'ME'},
    6: {'name': 'Saturn', 'file': 'saturn2.png', 'code': 'SA'},
    7: {'name': 'Neptune', 'file': 'neptune.png', 'code': 'NE'},
    8: {'name': 'Uranus', 'file': 'uranus.png', 'code': 'UR'},
    9: {'name': 'Asteroid', 'file': 'asteroid.png', 'code': 'AS'},
    10: {'name': 'Black Hole', 'file': 'black-hole.png', 'code': 'BL'},
    11: {'name': 'Star Destroyer', 'file': 'star-destroyer.png', 'code': 'ST'}
}

# Get the total number of objects
n_objects = len(objects)

# Define a list to store background images
bg = []
bg.append(Image.open(r"../img/background/bg1.jpg"))
bg.append(Image.open(r"../img/background/bg2.jpg"))
bg.append(Image.open(r"../img/background/bg3.jpg"))

# Set the directory path for object images
im_dir = '../img/objects'

# Loop through each object in the objects dictionary
for class_id, values in objects.items():
    # Open and convert the image file for the current planet
    png_file = Image.open(os.path.join(im_dir, values['file'])).convert('RGBA')
    
    # Crop the image to a square shape using the maximum dimension
    png_file = png_file.crop((0, 0, np.max(png_file.size), np.max(png_file.size)))
    
    # Store the processed image in the objects dictionary
    objects[class_id]['image'] = png_file
    
# Function to get a random background image and crop it
def get_bg(bg): 
    id_bg = np.random.randint(0, 3)  # Randomly select a background image index
    bg_tmp = bg[id_bg]  # Get the selected background image
    (w, h) = bg_tmp.size  # Get the dimensions of the background image
    x1 = np.random.randint(0, w - bg_size)  # Randomly choose x-coordinate for cropping
    y1 = np.random.randint(0, h - bg_size)  # Randomly choose y-coordinate for cropping
    bg_tmp = bg_tmp.crop((x1, y1, x1 + bg_size, y1 + bg_size))  # Crop the background image
    return bg_tmp

# Function to overlay a planet onto the background image
def put_object(obj, bg_tmp):
    h = obj.size[0]  # Get the size of the object image
    if h >= bg_size:
        scale = 0.4 * np.random.random() + 0.1  # Randomly choose a scale for large object
    else:
        scale = 0.7 * np.random.random() + 0.1  # Randomly choose a scale for small object
    h = np.int32(scale * h)  # Calculate the new size of the object image based on the scale
    p = obj.resize((h, h))  # Resize the object image
    h_bg = bg_tmp.size[0]  # Get the size of the background image
    x = np.random.randint(0, h_bg - h)  # Randomly choose x-coordinate for placing the object
    y = np.random.randint(0, h_bg - h)  # Randomly choose y-coordinate for placing the object
    bg_tmp.paste(p, (x, y), mask=p)  # Paste the object onto the background image
    return bg_tmp, x, y, h

# Function to create a single example for the dataset
def create_example():
    class_id = np.random.randint(0, n_objects)  # Randomly choose a class ID
    bg_tmp = get_bg(bg)  # Get a random background image
    plan_im = objects[class_id]['image']  # Get the image of the chosen class
    img, x, y, h = put_object(plan_im, bg_tmp)  # Overlay the object on the background
    img = img.resize((im_size, im_size))  # Resize the image to the desired size
    x1 = np.float32(x) / bg_size  # Normalize x-coordinate of the object's position
    y1 = np.float32(y) / bg_size  # Normalize y-coordinate of the object's position
    h = np.float32(h) / bg_size  # Normalize size of the object
    return img, class_id, x1, y1, h

# Function to create the dataset
def create_dataset(set_size):
    dataset = []
    for i in tqdm(range(set_size)):
        image, class_id, x1, y1, h = create_example()
        dataset.append([image, class_id, x1, y1, h])
    return dataset

bg_size = 800  # Size of the background image
im_size = 144  # Size of the resized images

print('Generating training set...')
dataset = create_dataset(1000)  # Create a dataset of 1000 examples

# display first 100 images
for i in range(10):
    for j in range(10):
        plt.subplot(10,10,10*i+j+1)
        plt.imshow(dataset[10*i+j][0])
        plt.axis('off')

## 3. Prepare for YOLO transfer

Here we prepare the dataset for the transfer to YOLOv5:
 - in the first cell we name and save the images into the `images` folder and produce the corresponding `txt` file containing labels, ie class id and bounding box characteristics
 - in the subsequent cells the images and labels sets are split and placed in train/val/test folders

In [ ]:
def dataset_to_yolo(dataset):
    #for i,data in enumerate(dataset):
    print("Prepare the dataset for the transfer to YOLOv5")
    for i in tqdm(range(len(dataset))):
        data = dataset[i]
        data[0].save(f'data/images/{i:05d}.png')
        c,x1,y1,h = data[1:]
        classname=objects[c]['name']
        #print(classname)
        tmp = np.array([c,x1+h/2,y1+h/2,h,h])
        np.savetxt(f'data/labels/{i:05d}.txt',tmp.reshape([1,5]),fmt='%d %1.4f %1.4f %1.4f %1.4f')
        

# Create a folder structure for YOLOv5 training
if not os.path.exists('data'):
    for folder in ['images', 'labels']:
        for split in ['train', 'val', 'test']:
            os.makedirs(f'data/{folder}/{split}')
dataset_to_yolo(dataset)

In [ ]:
len(dataset)
index = np.arange(len(dataset))
np.random.shuffle(index)

list_files=glob.glob("data/images/*.png") 
list_files.sort()
#print(list_files)

array_files=np.array(list_files)
np.random.shuffle(array_files)

In [ ]:
print("Split and move the images and files in train/test/val folders")

train_size = int(len(array_files)*0.7)
val_size=int(len(array_files)*0.15)

for i, image_path in enumerate(array_files):
    label_path = image_path.replace('.png', '.txt').replace('images','labels')
        
    # Split into train, val, or test
    if i < train_size:
        split = 'train'
    elif i < train_size + val_size:
        split = 'val'
    else:
        split = 'test'
   
    image_name=image_path.split('/')[-1]
    
    target_image_name = f'data/images/{split}/{image_name}'
    target_label_name = f'data/labels/{split}/{image_name.replace("png","txt")}'
    
    shutil.copy(image_path, target_image_name)
    shutil.copy(label_path, target_label_name)

## 4. run YOLOv5 transfer script

Once preprocessing is done, transfer learning and prediction can be performed using the command

In [ ]:
!python /usr/src/app/train.py --data spacequest.yaml --weights yolov5s.pt --epochs 20 --batch 4 --freeze 10

Validation examples can be found in the `runs/train/exp/` folder

In [ ]:
Image.open("../../runs/train/exp/val_batch2_pred.jpg")

Finally we generate a new example and apply detection with and without transfer learning

In [ ]:
im_size = 400
image, class_id, x1, y1, h = create_example()
image.save("example.png")
image

In [ ]:
!python /usr/src/app/detect.py --data spacequest.yaml --weights /usr/src/app/runs/train/exp/weights/best.pt --source example.png

with transfer learning

In [ ]:
# to dipslay the result of the detection you may have to adapt 
# the file path to match the output of the previous cell
Image.open("../../runs/detect/exp9/example.png")

In [ ]:
!python /usr/src/app/detect.py --data spacequest.yaml --weights yolov5s.pt --source example.png

without transfer learning

In [ ]:
Image.open("../../runs/detect/exp/example.png")